# GANomaly Notebook Results

## Initial Configurations

### Libraries import

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import umap
import cv2
from IPython.display import display
from ipywidgets import interact, IntSlider

sys.path.append("../../")

In [2]:
from utils.metrics import shapiroWilks_test, dagostinoPearson_test, bartlett_test, levene_test, fOneWay_test
from utils.savers import generate_qq_plot

### Experiment selection

In [4]:
experiment_id = "0007"
root_path = "/home/jefelitman/Saved_Models/Anomaly_parkinson/"
for i in sorted(os.listdir("/home/jefelitman/Saved_Models/Anomaly_parkinson/")):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

'/home/jefelitman/Saved_Models/Anomaly_parkinson/0007_Ganomaly_2D-64x64x64'

## Quantitative metrics

### Metrics loading

In [5]:
train_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/train.csv"))
data = train_metrics
np.max(data, axis=0), np.min(data, axis=0), data.to_numpy()[-1]

(epoch          9999.000000
 gen_error        15.544238
 disc_error        7.823781
 accuracy          0.930556
 precision         0.000000
 recall                 NaN
 specificity       0.930556
 f1_score          0.000000
 auc               0.000000
 dtype: float64,
 epoch          0.000000
 gen_error      1.018160
 disc_error     0.588961
 accuracy       0.486111
 precision      0.000000
 recall              NaN
 specificity    0.486111
 f1_score       0.000000
 auc            0.000000
 dtype: float64,
 array([9.99900000e+03, 1.08318555e+00, 7.71247387e+00, 7.77777791e-01,
        0.00000000e+00,            nan, 7.77777791e-01, 0.00000000e+00,
        0.00000000e+00]))

In [6]:
test_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/test.csv"))
data = test_metrics
np.max(data, axis=0), np.min(data, axis=0), data.to_numpy()[-1]

(epoch          9999.000000
 accuracy          0.932692
 precision         1.000000
 recall            0.920455
 specificity       1.000000
 f1_score          0.958580
 auc               1.000000
 dtype: float64,
 epoch          0.000000
 accuracy       0.076923
 precision      0.214286
 recall         0.034091
 specificity    0.312500
 f1_score       0.058824
 auc            0.204545
 dtype: float64,
 array([9.99900000e+03, 5.86538434e-01, 1.00000000e+00, 5.11363626e-01,
        1.00000000e+00, 6.76691729e-01, 1.00000000e+00]))

### Train Graphics

In [10]:
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
portions = 500
rows = 5
columns = 4
for metric in ["gen_error", "disc_error", "accuracy", "specificity"]:
    fig, axs = plt.subplots(rows, columns, figsize=(30, 25))
    fig.suptitle('{} over Epochs'.format(metric))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(train_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'train_{}.png'.format(metric)
    fig.savefig(path+filename)
    plt.close(fig)

### Test graphics

In [11]:
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
portions = 500
rows = 5
columns = 4
for metric in test_metrics.columns[1:]:
    fig, axs = plt.subplots(rows, columns, figsize=(30, 25))
    fig.suptitle('{} over Epochs'.format(metric))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(test_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'test_{}.png'.format(metric)
    fig.savefig(path+filename)
    plt.close(fig)

## Qualitative metrics

### Errors loading

In [13]:
base_path = os.path.join(experiment_folder, "outputs/errors/")
for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            if all_data not in globals().keys():
                globals()[all_data] = np.r_[[]]
            
            errors = np.load(os.path.join(base_path, t, m, c+".npy"))
            globals()["{}_{}_{}".format(m, t, c)] = errors
            globals()[all_data] = np.concatenate([globals()[all_data], errors])

### Train means and stds

In [14]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        data = globals()["train_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

encoder error (normal): 0.0025080834901826973 +- 0.0014001932501587794

contextual error (normal): 0.0206514958343986 +- 0.0030040974182617957

adversarial error (normal): 0.043870693202027015 +- 0.013637413307293152



### Train statistical tests

In [15]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        print("-------------------------------------------------")
        data = globals()["train_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

-------------------------------------------------
D'agostino-Pearson test for encoder (normal): 3.5184963167717445e-06
Shapiro-Wilks test for encoder (normal): 2.344929725950351e-06
Barlett test for encoder (normal): 3.915935961124386e-183
Levene test for encoder (normal): 9.499230984278309e-20
F1 test for encoder (normal): 0.07608029703921523
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for contextual (normal): 0.022296212328180826
Shapiro-Wilks test for contextual (normal): 0.004684805404394865
Barlett test for contextual (normal): 4.6360496877840635e-161
Levene test for contextual (normal): 2.732088321897578e-21
F1 test for contextual (normal): 0.8729308026138793
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for adversarial (normal): 0.05704069250220148
Shapiro-Wilks test for adversarial (normal): 0.004151544999331236
Barlett 

### Test means and stds

In [16]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["test_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

encoder error (normal): 0.002167530466977041 +- 0.0008331999687415564
encoder error (abnormal): 0.31252001717009326 +- 0.12864213281290274

contextual error (normal): 0.02123068564105779 +- 0.0021946933317612237
contextual error (abnormal): 0.16488883475011046 +- 0.020543156872943457

adversarial error (normal): 0.052566925529390574 +- 0.015829520480799407
adversarial error (abnormal): 0.6470565812831576 +- 0.21261434485833536



### Test statistical tests

In [17]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        print("-------------------------------------------------")
        data = globals()["test_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

-------------------------------------------------
D'agostino-Pearson test for encoder (normal): 0.0012233838688052742
Shapiro-Wilks test for encoder (normal): 0.014776092953979969
Barlett test for encoder (normal): 9.052215604053039e-43
Levene test for encoder (normal): 6.050504954439661e-05
F1 test for encoder (normal): 0.1280558186804157
-------------------------------------------------
-------------------------------------------------
D'agostino-Pearson test for encoder (abnormal): 0.004594977283238486
Shapiro-Wilks test for encoder (abnormal): 8.364874520339072e-05
Barlett test for encoder (abnormal): 8.457663246366759e-51
Levene test for encoder (abnormal): 5.630032962350611e-28
F1 test for encoder (abnormal): 0.017916308170033913
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for contextual (normal): 0.07247774670604946
Shapiro-Wilks test for contextual (normal): 0.03729066997766495
Barlett test for con

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


### All data means and stds

In [18]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["all_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

encoder error (normal): 0.0024461647586907598 +- 0.0013219508351728952
encoder error (abnormal): 0.31252001717009326 +- 0.12864213281290274

contextual error (normal): 0.020756803071972998 +- 0.0028826078034521227
contextual error (abnormal): 0.16488883475011046 +- 0.020543156872943457

adversarial error (normal): 0.04545182635245675 +- 0.014455913931924264
adversarial error (abnormal): 0.6470565812831576 +- 0.21261434485833536



### All data statistical tests

In [19]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        print("-------------------------------------------------")
        data = globals()["all_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

-------------------------------------------------
D'agostino-Pearson test for encoder (normal): 5.5146320522314296e-08
Shapiro-Wilks test for encoder (normal): 1.3985616931222467e-07
Barlett test for encoder (normal): 3.1179559469854234e-223
Levene test for encoder (normal): 1.7334555115999248e-23
F1 test for encoder (normal): 0.18247039029239423
-------------------------------------------------
-------------------------------------------------
D'agostino-Pearson test for encoder (abnormal): 0.004594977283238486
Shapiro-Wilks test for encoder (abnormal): 8.364874520339072e-05
Barlett test for encoder (abnormal): 2.084687883888159e-56
Levene test for encoder (abnormal): 8.463638305752743e-22
F1 test for encoder (abnormal): 0.004862282976829902
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for contextual (normal): 0.025634028906761402
Shapiro-Wilks test for contextual (normal): 0.005384721327573061
Barlett tes

### Train graphics

#### QQ plots

In [20]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        generate_qq_plot(globals()["train_{}_{}".format(t,c)], save_path, "train_{}_{}".format(t,c))

### Test graphics

#### Box plots

In [21]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "test_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'test_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [22]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        generate_qq_plot(globals()["test_{}_{}".format(t,c)], save_path, "test_{}_{}".format(t,c))

### All data graphics

#### Box plots

In [23]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "all_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'all_data_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [24]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        generate_qq_plot(globals()["all_{}_{}".format(t,c)], save_path, "all_data_{}_{}".format(t,c))

## Visual results

### UMAP latent vectors

In [25]:
base_path = os.path.join(experiment_folder, "outputs/latent_vectors/")
for t in ["in", "out"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            if all_data not in globals().keys():
                globals()[all_data] = []
            
            data = "{}_{}_{}".format(m, t, c)
            globals()[data] = []
            path = os.path.join(base_path, t, m, c)
            for file in sorted(os.listdir(path)):
                vector = np.load(os.path.join(path, file))
                globals()[data].append(vector)
                globals()[all_data].append(vector)
            globals()[data] = np.r_[globals()[data]]
for t in ["in", "out"]:
    for c in ["normal", "abnormal"]:
        all_data = "all_{}_{}".format(t, c)
        globals()[all_data] = np.r_[globals()[all_data]]

In [26]:
save_path = os.path.join(experiment_folder, "outputs/graphics/visuals/")
for t in ["in", "out"]:
    for m in ["train", "test", "all"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        globals()["{}_mapped_{}".format(m, t)] = umap.UMAP(random_state = 8128).fit_transform(np.concatenate(
                [globals()["{}_{}_{}".format(m, t, c)] for c in classes], axis=0
            )
        )
        
        divisor = globals()["{}_{}_normal".format(m, t)].shape[0]
        plt.scatter(globals()["{}_mapped_{}".format(m, t)][:divisor,0],
            globals()["{}_mapped_{}".format(m, t)][:divisor,1], color="blue", s=5, label="Normal"
        )
        if "abnormal" in classes:
            plt.scatter(globals()["{}_mapped_{}".format(m, t)][divisor:,0],
                globals()["{}_mapped_{}".format(m, t)][divisor:,1], color="red", s=5, label="Abnormal"
            )
        plt.legend()
        plt.title("UMAP for {}put data in {}".format(t, m))
        filename = '{}_umap_{}put.png'.format(m, t)
        plt.savefig(save_path+filename)
        plt.close()

### Video comparision

In [33]:
video_index = 0
mode = "test"
video_class = "normal"

base_path = os.path.join(experiment_folder, "outputs/samples/")
for t in ["real", "fake", "substraction"]:
    path = os.path.join(base_path, t, mode, video_class)
    video = sorted(os.listdir(path))[video_index]
    data = "video_{}".format(t)
    globals()[data] = []
    for frame in sorted(os.listdir(os.path.join(path, video))):
        globals()[data].append(cv2.cvtColor(
            cv2.imread(
                os.path.join(path, video, frame)
            ), cv2.COLOR_BGR2RGB
        ))
    globals()[data] = np.r_[globals()[data]]  
    
assert video_real.shape[0] == video_fake.shape[0] == video_substraction.shape[0]

widget = IntSlider(min=1, max=video_real.shape[0], step=1)

interact(lambda frame: plt.imshow(video_real[frame-1]), frame=widget)
interact(lambda frame: plt.imshow(video_fake[frame-1]), frame=widget)
interact(lambda frame: plt.imshow(video_substraction[frame-1]), frame=widget)

interactive(children=(IntSlider(value=1, description='frame', max=64, min=1), Output()), _dom_classes=('widget…

interactive(children=(IntSlider(value=1, description='frame', max=64, min=1), Output()), _dom_classes=('widget…

interactive(children=(IntSlider(value=1, description='frame', max=64, min=1), Output()), _dom_classes=('widget…

<function __main__.<lambda>(frame)>